In [1]:
!git clone https://github.com/quicksilverTrx/llm-forge

Cloning into 'llm-forge'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 63 (delta 16), reused 49 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 34.97 KiB | 11.66 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [2]:
!ls

llm-forge  sample_data


In [3]:
!pwd

/content


In [4]:
%cd llm-forge

/content/llm-forge


In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
!pwd

/content/llm-forge/src/model


In [7]:
%cd src/model

/content/llm-forge/src/model


In [9]:
def report_gpu_usage(stage):
    allocated = torch.cuda.memory_allocated() / 1024**2  # Convert to MB
    reserved = torch.cuda.memory_reserved() / 1024**2  # Convert to MB
    print(f"[{stage}] GPU Memory Allocated: {allocated:.2f} MB")
    print(f"[{stage}] GPU Memory Reserved: {reserved:.2f} MB\n")

In [11]:
from transformers import GPT2Model
import torch

In [12]:
report_gpu_usage("before model load")

[before model load] GPU Memory Allocated: 0.00 MB
[before model load] GPU Memory Reserved: 0.00 MB



In [13]:
from transformers import GPT2Model, GPT2Tokenizer
import sys
import os
sys.path.append('/content/llm-forge/')
import torch
from torch.optim import AdamW
from src.attention.optimizations.grouped_query_attention import GroupedQueryAttention, GQAConfig
from model_patcher import patch_attention_in_gpt2, add_rope_to_attention, remove_position_embeddings
from train_utils import  train_epoch, evaluate, prepare_wikitext2, LanguageModelingWrapper



In [14]:
report_gpu_usage("initial")

[initial] GPU Memory Allocated: 0.00 MB
[initial] GPU Memory Reserved: 0.00 MB



In [15]:


model = GPT2Model.from_pretrained('gpt2')
model.to('cuda')  # Load to GPU

# Print model parameter count and estimated size
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params}")
print(f"Model size in memory (FP32): {num_params * 4 / (1024**2):.2f} MB")
print(f"Model size in memory (FP16): {num_params * 2 / (1024**2):.2f} MB")
report_gpu_usage("after model load")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Number of parameters: 124439808
Model size in memory (FP32): 474.70 MB
Model size in memory (FP16): 237.35 MB
[after model load] GPU Memory Allocated: 487.47 MB
[after model load] GPU Memory Reserved: 542.00 MB



In [16]:
# Delete the model
del model

# Free up unused GPU memory
torch.cuda.empty_cache()

In [17]:
# Load base model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = GPT2Model.from_pretrained('gpt2')
gpt2_config = base_model.config
# 1. Patch single attention variant
gqa_config = GQAConfig(
        num_query_heads=gpt2_config.n_head,  # 12 for GPT-2 small
        num_kv_heads=gpt2_config.n_head //3,  # Reduce KV heads by factor of 3
        head_dim=gpt2_config.n_embd // gpt2_config.n_head,  # 768/12 = 64
        dropout=gpt2_config.attn_pdrop
    )

print(gqa_config)

flash_model = patch_attention_in_gpt2(base_model, GroupedQueryAttention,gqa_config)


final_model = flash_model
final_model = LanguageModelingWrapper(final_model)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

final_model.to(device)


GQAConfig(num_query_heads=12, num_kv_heads=4, head_dim=64, dropout=0.1, max_seq_length=8192, use_bias=False)
Using random initialization for layer 0
Using random initialization for layer 1
Using random initialization for layer 2
Using random initialization for layer 3
Using random initialization for layer 4
Using random initialization for layer 5
Using random initialization for layer 6
Using random initialization for layer 7
Using random initialization for layer 8
Using random initialization for layer 9
Using random initialization for layer 10
Using random initialization for layer 11


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

LanguageModelingWrapper(
  (base_model): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GroupedQueryAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=256, bias=False)
          (v_proj): Linear(in_features=768, out_features=256, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
          (attention): ScaledDotProductAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActi

In [18]:
# Print model parameter count and estimated size
num_params = sum(p.numel() for p in final_model.parameters())
print(f"Number of parameters: {num_params}")
print(f"Model size in memory (FP32): {num_params * 4 / (1024**2):.2f} MB")
print(f"Model size in memory (FP16): {num_params * 2 / (1024**2):.2f} MB")
report_gpu_usage("after model load")

Number of parameters: 114968832
Model size in memory (FP32): 438.57 MB
Model size in memory (FP16): 219.29 MB
[after model load] GPU Memory Allocated: 439.33 MB
[after model load] GPU Memory Reserved: 552.00 MB



In [19]:
# Prepare data
train_loader, eval_loader = prepare_wikitext2(tokenizer,batch_size=8, seq_length=256)

optimizers =  AdamW(final_model.parameters(), lr=5e-5)

for epoch in range (100):
    train_epoch(final_model, train_loader, optimizers, device)
    evaluate(final_model, eval_loader, device)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/685k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.07M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/618k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

...

training loss :  5.570067882537842


Training:   1%|          | 37/4590 [00:19<37:15,  2.04it/s]

training loss :  6.124176025390625


Training:   1%|          | 37/4590 [00:19<40:23,  1.88it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 231.06 MiB is free. Process 3266 has 14.52 GiB memory in use. Of the allocated memory 13.93 GiB is allocated by PyTorch, and 467.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
num_params = sum(p.numel() for p in final_model.parameters())
print(f"Number of parameters: {num_params}")
print(f"Model size in memory (FP32): {num_params * 4 / (1024**2):.2f} MB")
print(f"Model size in memory (FP16): {num_params * 2 / (1024**2):.2f} MB")

Number of parameters: 114968832
Model size in memory (FP32): 438.57 MB
Model size in memory (FP16): 219.29 MB


In [10]:
!nvidia-smi

Wed Jan 15 05:21:08 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0              28W /  70W |  14089MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--